# Tutorial - Interactive Plots with IPython Widgets
This notebook shows how to use [IPython Widgets](http://nbviewer.ipython.org/github/ipython/ipython/blob/8f4dcac780723bb91eeda805c04eb81b687e4b38/examples/Interactive%20Widgets/Index.ipynb) to interactively plot data using IBM Knowledge Anyhow Workbench. The notebook is based on the [Backtesting application](http://www.eurexchange.com/vstoxx/app2/) provided by Eurex. It is meant to show functionality rather than providing an actual backtesting application. The notebook shows how to:
* get data from SoftLayer Object Storage
* plot the data using Python
* interact with the data using IPython Widgets

After running all the cells in this notebook, it can be used in Dashboard Mode by selecting 'View' from the menu bar and then clicking on 'Dashboard Mode'. Viewing this notebook in Dashboard Mode will hide everything in this notebook except the interactive widgets so the user can interactively plot the data.

## Works Cited

* Eurex Exchange, “Backtesting of VSTOXX Strategies” from http://eurexchange.com/advanced-services/vstoxx/06_Backtesting_Application.html
* IPython, Interactive Widget Examples from the [documentation](http://nbviewer.ipython.org/github/ipython/ipython/blob/8f4dcac780723bb91eeda805c04eb81b687e4b38/examples/Interactive%20Widgets/Custom%20Widget%20-%20Hello%20World.ipynb)

## Import SoftLayer Object Storage Credentials File

You must provide credentials when connecting to SoftLayer Object Storage.  Credentials have been made available in a [JSON](http://json.org/) file.  **[Click here to import the credentials file](/tutorials/eurex/sl_object_store_credentials.json)** into your workbench.

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You only need to import the credentials file once.  Once you import the credentials file into your workbench, you can re-use the file from other notebooks to connect to the object store.</div>

## Connect to the Object Store

This notebook makes use of the [SoftLayer Object Storage Python Client](https://github.com/softlayer/softlayer-object-storage-python) package to connect to SoftLayer Object Storage.  You must install this package using the `pip` package installer as shown in the following code cell:

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You only need to install this package once.  It will persist in your workbench.</div>

In [ ]:
!pip install softlayer-object-storage

Import the [SoftLayer Object Storage Python Client](https://github.com/softlayer/softlayer-object-storage-python) to interact with SoftLayer Object Storage. Also import the [JSON module](https://docs.python.org/2/library/json.html#) to get the credentials from the provided JSON file. Then create a connection to your object store using the credentials from the JSON file.

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You can re-use the code below in other notebooks to create a connection to the object store.</div>

In [ ]:
import object_storage
import json

with file('/resources/sl_object_store_credentials.json') as f:
    sl_key = json.load(f)
sl_storage = object_storage.get_client(sl_key['username'], sl_key['api_key'], sl_key['server'])

## Get the EURO STOXX 50 and VSTOXX Data from the Object Store

Your object store contains historical data for the VSTOXX and the EURO STOXX 50 indices. Specifically, the data represents the daily closing values:
* from 01/01/1999 to the last available trading day (when the data was pulled) for the VSTOXX index
* from 12/31/1986 to the last available trading day (when the data was pulled) for the EURO STOXX 50 index

As highlighted in the [SoftLayer Object Storage tutorial](/tutorials/eurex/Tutorial%20-%20SoftLayer%20Object%20Storage.ipynb), each data set has been loaded into its own SoftLayer [container](http://sldn.softlayer.com/article/introduction-object-storage#Container) [object](http://sldn.softlayer.com/article/introduction-object-storage#Object). You can use the following Python modules to facilitate the extraction of the data in your object store:

* [StringIO module](https://docs.python.org/2/library/stringio.html) to read and write files
* [Pandas module](http://pandas.pydata.org/) to facilitate data analysis
* [datetime module](https://docs.python.org/2/library/datetime.html) to filter data

In [ ]:
import StringIO
import pandas
import datetime

Create a DataFrame from the VSTOXX data.

In [ ]:
vs_container_object = sl_storage.get_object("smarterfinancemarketplace","eurex/tutorials/vstoxx/vs.txt")
vs_data = StringIO.StringIO(vs_container_object.read())
vs_df = pandas.DataFrame.from_csv(vs_data, sep=',', parse_dates=True)
vs_df.sort_index(inplace=True)

Create a DataFrame from the EURO STOXX 50 data.

In [ ]:
es_container_object = sl_storage.get_object("smarterfinancemarketplace","eurex/tutorials/vstoxx/es.txt")
es_data = StringIO.StringIO(es_container_object.read())
es_df = pandas.DataFrame.from_csv(es_data, sep=';', parse_dates=True)
es_df.sort_index(inplace=True)

Create a DataFrame of the major indices only (V2TX from the VSTOXX and SX5E from the EURO STOXX 50). Additionally, remove any EURO STOXX 50 data prior to 1999 so that it's consistent with the VSTOXX data and sort the data by date.

In [ ]:
import datetime

assets_df = pandas.DataFrame({'SX5E' : es_df['SX5E'][es_df.index > datetime.datetime(1999, 1, 1)],
                              'V2TX' : vs_df['V2TX'][vs_df.index > datetime.datetime(1999, 1, 1)]})
assets_df.sort_index(inplace=True)

## Define Python Functions to Filter and Plot the Data

You can now define Python functions which will use the above data to interactively filter and plot the data as a [derivative](http://en.wikipedia.org/wiki/Derivative). The derivative will be based on three criteria which will be maintained in the following 3 variables:
* date_range - the date range for the investment
* assets - the set of underlying assets in the derivative
* asset_allocation - the allocation of each asset in the derivative

The initial state of the derivative will be based solely on the VSTOXX index. Thus, define the set of assets to 'V2TX' and an allocation of 100% to the VSTOXX asset. The initial data range is set to the full range of data (1/1/99 to 12/28/12).

In [ ]:
from sets import Set

def set_defaults():
    global date_range, assets, asset_allocation
    
    date_range = ['1999-01-02','2012-12-28']
    assets = Set(['V2TX'])
    asset_allocation = {
        'V2TX': 1,
        'SX5E': 0
    }

Define a function through which the underlying assets can be updated. The function does the following:
* adds new underlying assets when the user selects an asset from a dropdown widget
* removes an underlying asset when the user de-selects an asset from a dropdown widget (after checking it's not selected in any other dropdown widget)
* enables the slider widget (to adjust asset allocation) if 2 unique assets have been selected (no more than 2 assets can be selected)
* disables the slider widget and resets allocation if only 1 asset is selected

In [ ]:
def update_asset_list(name, old, new):
    if new != '-----':
        assets.add(new)
    if old != '-----' and asset1_dropdown.value != old and asset2_dropdown.value != old:
        assets.remove(old)
    if len(assets) == 2:
        asset2_allocation_slider.disabled = False
    else:
        asset2_allocation_slider.disabled = True
        if asset2_allocation_slider.value != 0:
            asset1_allocation_value.value = '100%'
            asset2_allocation_slider.value = 0
            return
    update_asset_allocation(None, asset2_allocation_slider.value)

Define a function through which the investment period can be updated. The function sets the start date and end date of the investment period based on the input values of date selection widgets.

In [ ]:
def update_date_range():
    if start_date_input.value > end_date_input.value:
        print '\033[0;31mEnd date must be greater than start date.'
        return
    if start_date_input.value:
        date_range[0] = start_date_input.value
    if end_date_input.value:
        date_range[1] = end_date_input.value
    render_chart()

Define a function through which the asset allocation can be updated. The allocation percentages are updated through a slider widget associated with the 2nd underlying asset. The function does the following:
* calculate (as a decimal) the allocation percentage of the 1st underlying asset based on the slider value for the 2nd asset
* calculate (as a decimal) the allocation percentage of the 2nd underlying asset based on the slider value (after checking the 2nd asset has not been de-selected)
* update the displayed allocation percentage for the 1st underlying asset

In [ ]:
def update_asset_allocation(name, new):
    asset_allocation[asset1_dropdown.value] = float((100-new))/float(100)
    if asset2_dropdown.value != '-----' and asset2_dropdown.value != asset1_dropdown.value:
        asset_allocation[asset2_dropdown.value] = float(new)/float(100)
    asset1_allocation_value.value = '{}%'.format(100-new)
    render_chart()

Define a function to calculate absolute return values. The plot of the derivative (and its underlying assets) will show absolute return values.

In [ ]:
def absolute_return(s):
    start_value = s[0]
    return 100 * (s / start_value - 1)

Define a function which will plot the derivative and its underlying assets. The function does the following:
* clears any pre-existing plots
* creates a temporary dataframe with absolute return values for the selected underlying assets, for the selected investment period
* creates a new column in the dataframe which calculates the absolute return value for the derivative based on the allocation percentages for each underlying asset
* creates a plot of the dataframe

In [ ]:
%matplotlib inline

from IPython.display import clear_output

def render_chart():
    clear_output(wait=True)
    asset_list = list(assets)
    if len(asset_list) > 0:
        display_assets_df = assets_df[date_range[0]:date_range[1]][asset_list].apply(absolute_return)
        if len(assets) == 1:
            display_assets_df['Derivative'] = display_assets_df[asset_list[0]]*asset_allocation[asset_list[0]]
        else:
            display_assets_df['Derivative'] = display_assets_df[asset_list[0]]*asset_allocation[asset_list[0]] + display_assets_df[asset_list[1]]*asset_allocation[asset_list[1]]
        ax = display_assets_df.plot(rot=360, figsize=(18,9))
        ax.set_ylabel('Absolute Return(%)')

## Define and Create IPython Widgets to Interact with the Data

Define a custom DatePicker Widget for interactively updating the investment period. This custom widget come from the [IPython Widget examples](http://nbviewer.ipython.org/github/ipython/ipython/blob/8f4dcac780723bb91eeda805c04eb81b687e4b38/examples/Interactive%20Widgets/Custom%20Widget%20-%20Hello%20World.ipynb). The custom widget defines:
* the HTML which should be rendered
* an update function to handle updates from the Python functions
* an event and event handler to handle updates from the widget

In [ ]:
%%javascript
require(["widgets/js/widget", "widgets/js/manager"], function(widget, manager){
    var DatePickerView = IPython.DOMWidgetView.extend({
        render: function(){
            this.$date = $('<input />')
                .attr({
                    'type':'date',
                    'min':'1999-01-02',
                    'max':'2012-12-28'
                })
                .appendTo(this.$el);
        },
        update: function() {
            this.$date.val(this.model.get('value'));
            return DatePickerView.__super__.update.apply(this);
        },
        events: {"change": "handle_date_change"},
        handle_date_change: function(event) {
            var selected_date = new Date(this.$date.val());
            var min_date = new Date(this.$date.attr('min'));
            var max_date = new Date(this.$date.attr('max'));
            if(selected_date > min_date && selected_date < max_date) {
                this.model.set('value', this.$date.val());
                this.touch();
            }
        }
    });
    manager.WidgetManager.register_widget_view('DatePickerView', DatePickerView);
});

Define a class through which the custom widget can be instantiated.

In [ ]:
from IPython.html import widgets
from IPython.utils.traitlets import Unicode

class DateWidget(widgets.DOMWidget):
    _view_name = Unicode('DatePickerView', sync=True)
    value = Unicode(sync=True)

Define a function through which the date selection widgets will be created. The function creates a list of widgets through which the investment period can be updated. The widget list will include:
* HTML widgets to display text labels
* DateWidgets to select start and end dates

The widgets are stored as a list so that they can be styled later using the Container widget.

In [ ]:
def create_date_selection_widgets():
    global start_date_input, end_date_input
    dates_input_list = []

    start_date_label = widgets.HTML(value="<label>Investment date </label>")
    start_date_label.margin = '5px'
    start_date_input = DateWidget()
    
    dates_input_list.append(start_date_label)
    dates_input_list.append(start_date_input)

    end_date_label = widgets.HTML(value="<label>Maturity date </label>")
    end_date_label.margin = '5px'
    end_date_input = DateWidget()
    dates_input_list.append(end_date_label)
    dates_input_list.append(end_date_input)
    return dates_input_list

Define a function through which the asset selection widgets will be created. The function creates a list of widgets through which the underlying assets can be updated. The function also defines that updates to the widgets should be handled by the appropriate Python functions (update_asset_list & update_asset_allocation) so user updates are interactively shown. The widget list will include:
* Widget widgets to display text labels
* Dropdown to select underlying assets
* IntSlider to adjust asset allocation

The widgets are stored as a list so that they can styled later using the Container widget.

In [ ]:
def create_asset_selection_widgets():
    global asset2_allocation_slider, asset1_allocation_value, asset1_dropdown, asset2_dropdown
    assets_dropdown_list = []
    asset1_dropdown = widgets.Dropdown(description="Asset 1",
                                             options=['V2TX','SX5E'],
                                             value='V2TX')
    asset1_dropdown.on_trait_change(update_asset_list, 'value')
    
    asset1_allocation_label = widgets.HTML(value="<span>Ratio of Asset 1:</span>")
    asset1_allocation_label.margin = '10px'
    asset1_allocation_value = widgets.HTML(value="<span>100%</span>")
    asset1_allocation_value.margin = '10px'
    
    assets_dropdown_list.append(asset1_dropdown)
    assets_dropdown_list.append(asset1_allocation_label)
    assets_dropdown_list.append(asset1_allocation_value)
    
    asset2_dropdown = widgets.Dropdown(description="Asset 2",
                                       options=['-----','V2TX','SX5E'],
                                       value='-----')
    asset2_dropdown.on_trait_change(update_asset_list, 'value')
    
    asset2_allocation_slider = widgets.IntSlider(
        value=0,
        min=0,
        max=100,
        step=1,
        description='Ratio of Asset 2:',
        disabled = True
    )
    asset2_allocation_slider.margin = '5px'
    asset2_allocation_slider.msg_throttle = 1
    asset2_allocation_slider.on_trait_change(update_asset_allocation, 'value')
    
    assets_dropdown_list.append(asset2_dropdown)
    assets_dropdown_list.append(asset2_allocation_slider)

    return assets_dropdown_list

Define a function which will render the interactive widgets. The function creates a final set of widgets for titles. It uses Container widgets to control the layout of the interactive widgets. The function does the following:
* call the python functions which create the interactive widgets
* create new widgets to be used for titles
* embed all the widgets in containers
* display and style the containers
* set the default investment dates
* define the python function which will handle updates to the investment dates

In [ ]:
def render_widgets():
    dates_input_list = create_date_selection_widgets()
    assets_dropdown_list = create_asset_selection_widgets()
    
    date_section_title = widgets.HTML(value="<h3>Time Period</h3>")
    assets_section_label = widgets.HTML(value="<h3>Assets</h3>")
    
    overall_container = widgets.Box()
    dates_label_container = widgets.Box()
    dates_label_container.children = [date_section_title]

    dates_input_container = widgets.FlexBox()
    dates_input_container.orientation = 'horizontal'
    dates_input_container.children = dates_input_list

    assets1_input_container = widgets.FlexBox()
    assets1_input_container.orientation = 'horizontal'
    assets1_input_container.children = [assets_dropdown_list[0], assets_dropdown_list[1], assets_dropdown_list[2]]

    assets2_input_container = widgets.FlexBox()
    assets2_input_container.orientation = 'horizontal'
    assets2_input_container.children = [assets_dropdown_list[3], assets_dropdown_list[4]]

    overall_container.children = [dates_label_container, dates_input_container, assets_section_label,
                                  assets1_input_container, assets2_input_container]
    
    display(overall_container)    
    start_date_input.value = date_range[0]
    end_date_input.value = date_range[1]
    
    start_date_input.on_trait_change(update_date_range, 'value')
    end_date_input.on_trait_change(update_date_range, 'value')

## Render the Interactive Widgets

Run the Python functions defined above to set the default data values, render the interactive widgets, and render an initial chart.

Remember you can view this notebook in Dashboard Mode by selecting 'View' from the menu bar and then clicking on 'Dashboard Mode'. Viewing this notebook in Dashboard Mode will hide everything in this notebook except the interactive widgets so the user can interactively plot the data.

In [ ]:
asset1_dropdown = widgets.Dropdown(description="Asset 1",values=['V2TX','SX5E']) 

In [ ]:
from IPython.display import display

set_defaults()
render_widgets()
render_chart()

## Additional Tutorials
Additional tutorials for the IBM Knowledge Anyhow Workbench are available on our [Welcome](/pages/welcome) page.